### This is a script to run periodic model performance evaluation using the most recent 1 day of data

In [1]:
import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests, optuna, datetime, joblib, pytz, gcsfs
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft
import yfinance as yf
from google.cloud import storage
from io import BytesIO


from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

time0 = time.time()

bucket_path = 'gs://pmykola-streaming-projects/spg-stocks/data/'

pull_time = datetime.datetime.now()
pull_time = pull_time.astimezone(pytz.timezone('America/New_York'))
pull_time = pull_time.replace(tzinfo=None)
now_time = (str(pull_time.month) + '_' + 
str(pull_time.day) + '_' +
str(pull_time.hour) + ':'  +
str(pull_time.minute) + ':' +
str(pull_time.second))

In [2]:
datafiles = !gsutil ls gs://pmykola-streaming-projects/spg-stocks/data
start_file = [x for x in datafiles if ('data_start_' in x)]
datafiles = [x for x in datafiles if ('auto_data_last_' in x) & ('pull_time' in x)]
assert len(start_file) == 1
start_file = start_file[0]
start_file

'gs://pmykola-streaming-projects/spg-stocks/data/data_start_20221021.csv'

In [3]:
df = pd.read_csv(start_file)
df.Datetime = pd.to_datetime(df.Datetime)

df_new = pd.DataFrame(columns = df.columns)
for file in datafiles:
    temp_df = pd.read_csv(file)
    df_new = pd.concat([df_new, temp_df], axis=0)
    # remove duplicates
df_new.reset_index(inplace=True, drop=True)
df_new.drop_duplicates(inplace=True)
df_new.Datetime = pd.to_datetime(df_new.Datetime)
df_new.sort_values(by='Datetime')
df_new

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR
0,2022-12-06 09:30:00,39.240002,65.089996,49.399899,11229.620117,1836.604736,3999.110107,179.300003
1,2022-12-06 09:32:00,39.230000,65.110001,49.389999,11220.044922,1837.946777,4000.229980,179.360001
2,2022-12-06 09:34:00,39.189999,65.040001,49.389999,11208.439453,1837.892456,3996.840088,NaN
3,2022-12-06 09:36:00,39.230000,65.019997,49.419998,11215.238281,1838.812744,3998.750000,179.259995
4,2022-12-06 09:38:00,39.200001,65.029999,49.419998,11206.872070,1838.209717,3995.010010,179.220001
...,...,...,...,...,...,...,...,...
583,2022-12-08 15:52:00,39.195000,65.190002,49.150002,11059.537109,1813.920044,3954.919922,NaN
584,2022-12-08 15:54:00,39.224998,65.205002,49.150002,11075.905273,1814.733887,3960.989990,NaN
585,2022-12-08 15:56:00,39.235001,65.220001,49.154999,11077.916992,NaN,3961.790039,NaN
586,2022-12-08 15:58:00,39.259998,65.220001,49.180000,11080.791016,NaN,3962.909912,NaN


In [4]:
df = pd.read_csv(start_file)
df.Datetime = pd.to_datetime(df.Datetime)
df = pd.concat([df, df_new])
df.drop_duplicates(inplace=True)
df.sort_values(by='Datetime')
df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004
3,2022-10-21 09:36:00,34.709999,NaN,45.498699,10586.677734,1705.815674,3665.989990,NaN
4,2022-10-21 09:38:00,34.689999,NaN,45.463501,10583.666016,1704.521973,3662.169922,163.869995
...,...,...,...,...,...,...,...,...
583,2022-12-08 15:52:00,39.195000,65.190002,49.150002,11059.537109,1813.920044,3954.919922,NaN
584,2022-12-08 15:54:00,39.224998,65.205002,49.150002,11075.905273,1814.733887,3960.989990,NaN
585,2022-12-08 15:56:00,39.235001,65.220001,49.154999,11077.916992,NaN,3961.790039,NaN
586,2022-12-08 15:58:00,39.259998,65.220001,49.180000,11080.791016,NaN,3962.909912,NaN


In [5]:
df['time'] = df.Datetime.dt.time
df['date'] = df.Datetime.dt.date

df = df.fillna(method='ffill')
dayclose = df[df.time==datetime.time(15, 58, 0)]
dayopen = df[df.time==datetime.time(9, 30, 0)]
dayopen.reset_index(drop=True, inplace=True)
dayclose.reset_index(drop=True, inplace=True)
dayclose.sort_values(by='date')

df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998,09:30:00,2022-10-21
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998,09:32:00,2022-10-21
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004,09:34:00,2022-10-21
3,2022-10-21 09:36:00,34.709999,55.619999,45.498699,10586.677734,1705.815674,3665.989990,164.020004,09:36:00,2022-10-21
4,2022-10-21 09:38:00,34.689999,55.619999,45.463501,10583.666016,1704.521973,3662.169922,163.869995,09:38:00,2022-10-21
...,...,...,...,...,...,...,...,...,...,...
583,2022-12-08 15:52:00,39.195000,65.190002,49.150002,11059.537109,1813.920044,3954.919922,177.309998,15:52:00,2022-12-08
584,2022-12-08 15:54:00,39.224998,65.205002,49.150002,11075.905273,1814.733887,3960.989990,177.309998,15:54:00,2022-12-08
585,2022-12-08 15:56:00,39.235001,65.220001,49.154999,11077.916992,1814.733887,3961.790039,177.309998,15:56:00,2022-12-08
586,2022-12-08 15:58:00,39.259998,65.220001,49.180000,11080.791016,1814.733887,3962.909912,177.309998,15:58:00,2022-12-08


In [6]:
df.date.unique()[len(df.date.unique())-2]

datetime.date(2022, 12, 7)

In [7]:
df.date.max()

datetime.date(2022, 12, 8)

In [8]:
### now i wanna do feature engineering for all assets 

asset_list = ['Spx', 'Nasdaq', 'Russel', 'EMXC', 'EEMA', 'EEM', 'VTHR']

for asset in asset_list:
    
    df[asset + '_ret'] = 100*(df[asset]/df[asset].shift(1)-1)
    df['s_' + asset + '_ret_1prd'] = (100*(df[asset]/df[asset].shift(1)-1)).shift(1)
    df['s_' + asset + '_ret_2prd'] = (100*(df[asset]/df[asset].shift(2)-1)).shift(1)
    df['s_' + asset + '_ret_4prd'] = (100*(df[asset]/df[asset].shift(4)-1)).shift(1)
    # display(df.shape, df.head(5))

    df.loc[df.time < datetime.time(9, 32, 0), 's_' + asset + '_1prd'] = np.nan
    df.loc[df.time < datetime.time(9, 33, 0), 's_' + asset + '_2prd'] = np.nan
    df.loc[df.time < datetime.time(9, 35, 0), 's_' + asset + '_4prd'] = np.nan

    dayopen.rename(columns={asset:asset+'_open'}, inplace=True)
    # dayopen.head()
    dayclose.rename(columns={asset:asset+'_close'}, inplace=True)
    dayclose_l1 = dayclose.copy()
    dayclose_l2 = dayclose.copy()
    dayclose_l1[asset+'_close_l1'] = dayclose_l1[asset+'_close'].shift(1)
    dayclose_l2[asset+'_close_l2'] = dayclose_l2[asset+'_close'].shift(2)

    df = pd.merge(df, dayopen[['date', asset + '_open']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l1[['date', asset + '_close_l1']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l2[['date', asset + '_close_l2']], on=['date'], how='left')

    df['s_' + asset + '_ret_open'] = (100*(df[asset]/df[asset + '_open']-1)).shift(1)
    df['s_' + asset + '_ret_close1'] = (100*(df[asset]/df[asset + '_close_l1']-1)).shift(1)
    df['s_' + asset + '_ret_close2'] = (100*(df[asset]/df[asset + '_close_l2']-1)).shift(1)

    cols_todrop = [x for x in list(df.columns) if asset in x and 'ret' not in x]
    df.drop(columns = cols_todrop, inplace=True)

display(time.time() - time0, df.tail())

2.70161771774292

,Datetime,time,date,Spx_ret,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,Nasdaq_ret,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,Russel_ret,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,EMXC_ret,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,EEMA_ret,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,EEM_ret,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,VTHR_ret,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
6540,2022-12-08 15:52:00,15:52:00,2022-12-08,0.012391,0.002778,-0.012642,-0.118465,0.177331,0.533374,0.328046,0.008698,0.007489,-0.039246,-0.161546,0.482519,0.921701,0.387046,0.054991,-0.029120,-0.073030,-0.089016,-0.218336,0.280643,-0.002182,0.061080,0.000000,-0.061043,-0.142305,-0.142305,-0.081369,-0.101686,0.046039,0.007682,-0.046017,-0.046017,0.261588,1.023261,0.648758,0.025516,0.025532,-0.012751,-0.051015,-0.241847,0.940241,0.358569,0.0,0.0,-0.244962,-0.244962,0.647102,0.647102,0.362257
6541,2022-12-08 15:54:00,15:54:00,2022-12-08,0.153481,0.012391,0.015170,-0.047011,0.189743,0.545831,0.340478,0.148000,0.008698,0.016188,-0.087086,0.491260,0.930479,0.395778,0.044867,0.054991,0.025855,-0.024665,-0.163465,0.335789,0.052808,0.000000,0.061080,0.061080,-0.030508,-0.081311,-0.020338,-0.040668,0.023009,0.046039,0.053724,-0.030665,0.307747,1.069771,0.695095,0.076537,0.025516,0.051054,-0.025512,-0.216393,0.965997,0.384176,0.0,0.0,0.000000,-0.244962,0.647102,0.647102,0.362257
6542,2022-12-08 15:56:00,15:56:00,2022-12-08,0.020198,0.153481,0.165891,0.153228,0.343516,0.700150,0.494482,0.018163,0.148000,0.156712,0.117404,0.639987,1.079857,0.544364,0.000000,0.044867,0.099882,0.026779,-0.118671,0.380806,0.097699,0.010167,0.000000,0.061080,0.000000,-0.081311,-0.020338,-0.040668,0.023003,0.023009,0.069058,0.023009,0.330826,1.093026,0.718264,0.025499,0.076537,0.102072,0.089308,-0.140021,1.043274,0.461007,0.0,0.0,0.000000,-0.244962,0.647102,0.647102,0.362257
6543,2022-12-08 15:58:00,15:58:00,2022-12-08,0.028267,0.020198,0.173711,0.188907,0.363784,0.720489,0.514780,0.025944,0.018163,0.166190,0.182405,0.658266,1.098216,0.562626,0.000000,0.000000,0.044867,0.070733,-0.118671,0.380806,0.097699,0.050863,0.010167,0.010167,0.071254,-0.071152,-0.010173,-0.030505,0.000000,0.023003,0.046017,0.099766,0.353906,1.116281,0.741432,0.063713,0.025499,0.102056,0.153162,-0.114557,1.069039,0.486624,0.0,0.0,0.000000,0.000000,0.647102,0.647102,0.362257
6544,2022-12-08 16:00:00,16:00:00,2022-12-08,0.015143,0.028267,0.048471,0.214443,0.392153,0.748960,0.543192,0.010937,0.025944,0.044111,0.200892,0.684381,1.124445,0.588716,0.000000,0.000000,0.000000,0.099882,-0.118671,0.380806,0.097699,0.000000,0.050863,0.061035,0.122153,-0.020326,0.040684,0.020342,0.000000,0.000000,0.023003,0.092077,0.353906,1.116281,0.741432,0.000000,0.063713,0.089228,0.191392,-0.050918,1.133433,0.550647,0.0,0.0,0.000000,0.000000,0.647102,0.647102,0.362257


In [9]:
### do prediction ###

storage_client = storage.Client()
bucket_name='pmykola-streaming-projects'
model_path='spg-stocks/artifacts/en_model.pkl'

bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(model_path)
model_file = BytesIO()
blob.download_to_file(model_file)
trained_model=joblib.load(model_file)

this_day = df.loc[df.date == df.date.max()]
print(f'{this_day.shape[0]} observations this day')
X = this_day.copy()
X.drop(columns = ['Datetime',
                  'time', 
                  'date', 
                  'Spx_ret', 
                  'Nasdaq_ret', 
                  'Russel_ret', 
                  'EEMA_ret', 
                  'EEM_ret', 
                  'EMXC_ret', 
                  'VXUS_ret', 
                  'VTHR_ret'], 
                  inplace=True,
                  errors = 'ignore')

if(X.count().sum() < X.shape[1]):
    print(f'''There are {X.shape[1] - X.count().sum()} missing values. 
          There will be an error''')

y = this_day.VTHR_ret
y_hat = trained_model.predict(X)
#print(trained_model.predict(X))

print('Total time: ', time.time()-time0)

196 observations this day
Total time:  2.919823169708252


In [10]:
model_rmse = mean_squared_error(y, y_hat)
constant_rmse = mean_squared_error(y, np.zeros(len(y)))

performance = pd.DataFrame([[100*(r2_score(y, y_hat)), model_rmse, constant_rmse, 100*(1-model_rmse/constant_rmse)]], 
                           columns = ['R2', 'model_rmse', 'constant_rmse', 'rmse_improvement'])

file_name = 'm1_performance_' + \
str(df.date.max().year) + \
str(df.date.max().month) + \
str(df.date.max().day) + \
'_pull_time_' + \
now_time + \
'.csv'
# performance.to_csv('gs://pmykola-streaming-projects/spg-stocks/artifacts' + '/' + file_name)

In [11]:
performance.index = [df.date.max()]

In [12]:
performance

,R2,model_rmse,constant_rmse,rmse_improvement
2022-12-08,10.233361,0.007241,0.008077,10.356683


In [13]:
df.date.max()

datetime.date(2022, 12, 8)

In [16]:
performance.rename(columns={'R2':'r2'}, inplace=True)
performance

,r2,model_rmse,constant_rmse,rmse_improvement
2022-12-08,10.233361,0.007241,0.008077,10.356683
